In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("data.csv")

In [2]:
data.columns

Index(['label_p0_x', 'label_p0_y', 'label_p1_x', 'label_p1_y', 'label_p2_x',
       'label_p2_y', 'label_p3_x', 'label_p3_y', 'prediction_p0_x',
       'prediction_p0_y', 'prediction_p1_x', 'prediction_p1_y',
       'prediction_p2_x', 'prediction_p2_y', 'prediction_p3_x',
       'prediction_p3_y'],
      dtype='object')

In [3]:
boxA = data.filter(like = "label", axis = 1)
boxB = data.filter(like = 'prediction', axis = 1)
boxB = boxB.fillna(0)

In [4]:
def clip(subject_polygon, clip_polygon):
    def inside(point):
        cond1 = (clip_vertex_point[0]-clip_polygon_last_point[0])*(point[1]-clip_polygon_last_point[1])
        cond2 = (clip_vertex_point[1]-clip_polygon_last_point[1])*(point[0]-clip_polygon_last_point[0])
        return cond1 > cond2

    def computeIntersection():
        dc = [ clip_polygon_last_point[0] - clip_vertex_point[0], clip_polygon_last_point[1] - clip_vertex_point[1] ]
        dp = [point_s[0] -point_e[0], point_s[1] -point_e[1] ]
        n1 = clip_polygon_last_point[0] * clip_vertex_point[1] - clip_polygon_last_point[1] * clip_vertex_point[0]
        n2 = point_s[0] *point_e[1] - point_s[1] *point_e[0] 
        n3 = 1.0 / (dc[0] * dp[1] - dc[1] * dp[0])
        return np.array([(n1*dp[0] - n2*dc[0]) * n3, (n1*dp[1] - n2*dc[1]) * n3])

    outputList = subject_polygon
    clip_polygon_last_point = clip_polygon[-1]

    for clipVertex in clip_polygon:
        clip_vertex_point = clipVertex
        inputList = outputList
        outputList = []
        try:
            point_s= inputList[-1]
        except IndexError:
            break

        for subjectVertex in inputList:
            point_e = subjectVertex
            if inside(point_e):
                if not inside(point_s):
                    outputList.append(computeIntersection())
                outputList.append(point_e)
            elif inside(point_s):
                outputList.append(computeIntersection())
            point_s=point_e
        clip_polygon_last_point = clip_vertex_point
    return(list(map(list, zip(*outputList))))

In [5]:
def shoelace_formula_area(x,y):
    """
    x,y - coordinates
    """
    main_area = np.dot(x[:-1],y[1:]) - np.dot(x[1:], y[:-1])
    edge_terms = x[-1]*y[0] - y[-1]*x[0]
    return 0.5*abs(main_area + edge_terms)

In [6]:
results = []
for x,y in zip(boxA.values,boxB.values):
    points_label = list(zip(x[::2], x[1::2]))
    points_prediction = list(zip(y[::2], y[1::2]))
    intersection = clip(points_label, points_prediction)
    label_area = shoelace_formula_area(x[::2], x[1::2])
    prediction_area = shoelace_formula_area(y[::2], y[1::2])
    try:
        intersection_area = shoelace_formula_area(intersection[0], intersection[1])
    except IndexError:
        results.append(0)
        continue
    results.append(intersection_area/(label_area + prediction_area - intersection_area))

In [9]:
results

[0.9828619185319958,
 0.9784916297229584,
 0.9700129529607042,
 0.9776211819523805,
 0.9803975413236272,
 0.9877165031673658,
 0.9770460948860413,
 0.9856374655750745,
 0.9081445089746749,
 0,
 0,
 0,
 0,
 0.985829068221522,
 0,
 0.9725640876484161,
 0.980030190127262,
 0.9854283284515601,
 0.976626807153363,
 0.9724237291997144,
 0.9889137828742517,
 0,
 0.9834323923201935,
 0.976794464959044,
 0.9912647388933321,
 0.9908650835062721,
 0,
 0.992062687078529,
 0.9771168615397869,
 0.9743486892239827,
 0.982230964514802,
 0.9830708084472384,
 0.9764680153395857,
 0.9785632291060444,
 0.969252941557348,
 0.9781241837735781,
 0.9745068487173257,
 0.9696048646721576,
 0,
 0.9809132385536113,
 0.9846083596481366,
 0.9838327452876598,
 0.9255210556277839,
 0.9895728620798572,
 0.9816960216974129,
 0.9860126598480242,
 0.9811382455757004,
 0.984953969442557,
 0,
 0.9870762314162369,
 0.9788250786928535,
 0.9774914679055494,
 0.9813477296064795,
 0.9734618204600005,
 0.9730396545047897,
 0.979